# 🔍 Advanced Search

Fast Search is Relevance AI's most complex search endpoint. 
It combines functionality to search using vectors, exact text search with ability to boost your search results depending on your needs. The following demonstrates a few dummy examples on how to quickly add complexity to your search!

In [27]:
!pip install -q RelevanceAI[notebook]

In [28]:
## Let's use this CLIP popular model to encode text and image into same space https://github.com/openai/CLIP

!conda install --yes -c pytorch pytorch=1.7.1 torchvision cudatoolkit=11.0
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git

/bin/bash: conda: command not found
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-8opfrum1
  Running command git clone -q https://github.com/openai/CLIP.git /tmp/pip-req-build-8opfrum1


You can sign up/login and find your credentials here: https://cloud.relevance.ai/sdk/api
Once you have signed up, click on the value under `Authorization token` and paste it here

In [29]:

import pandas as pd
from relevanceai import Client

client = Client()


Activation token (you can find it here: https://cloud.relevance.ai/sdk/api )

Activation Token: ··········
Connecting to ap-southeast-2...
You can view all your datasets at https://cloud.relevance.ai/datasets/
Welcome to RelevanceAI. Logged in as 22d8b47fbed6cf1391f0.


## 🚣 Inserting data

We use a sample ecommerce dataset - with vectors `product_image_clip_vector_` and `product_title_clip_vector_` already encoded for us.

In [15]:

from relevanceai.utils.datasets import get_ecommerce_dataset_encoded

docs = get_ecommerce_dataset_encoded()
docs[0].keys()


dict_keys(['product_image', 'query', 'product_price', 'source', 'product_title', 'product_link', 'product_image_clip_vector_', 'product_title_clip_vector_', 'insert_date_', 'price', '_id'])

In [16]:
ds = client.Dataset('advanced_search_guide')
# ds.delete()
ds.upsert_documents(docs)

✅ All documents inserted/edited successfully.


In [17]:
ds.schema

{'insert_date_': 'date',
 'price': 'numeric',
 'product_image': 'text',
 'product_image_clip_vector_': {'vector': 512},
 'product_link': 'text',
 'product_price': 'text',
 'product_title': 'text',
 'product_title_clip_vector_': {'vector': 512},
 'query': 'text',
 'source': 'text'}

In [18]:
vector_fields = ds.list_vector_fields()
vector_fields

['product_image_clip_vector_', 'product_title_clip_vector_']

## Simple Text Search

In [8]:
results = ds.advanced_search(query="nike", 
                            fields_to_search=["product_title"], 
                             select_fields=['product_title']
                             )
pd.DataFrame(results['results'])

,product_title,_id,_relevance
0,Nike Mens Lunar Cypress Spikeless Golf Shoes,fb323476-a16d-439c-9380-0bac1e10a06d,6.755055
1,Nike Women's SQ Dymo STR8-FIT Driver,ff52b64a-0567-4181-8753-763da7044f2f,6.755055
2,Nike Women's 'Lunaracer+ 3' Mesh Athletic Shoe,0614f0a9-adcb-4c6c-939c-e7869525549c,6.755055
3,Nike SolarSoft Golf Grill Room Black Shoes,22871acd-fbc9-462e-8305-26df642c915c,6.755055
4,Nike Women's Lunar Duet Classic Golf Shoes,6f85d037-7621-45ee-b5dc-dd0e88c58d4a,6.755055
5,Nike Women's 'Lunaracer+ 3' Mesh Athletic Shoe,7baea34f-fb0a-47da-9edd-d920abddccf5,6.755055
6,Nike Ladies Lunar Duet Sport Golf Shoes,80210247-6f40-45be-8279-8743b327f1dc,6.755055
7,Nike Men's 'Lunarglide 6' Synthetic Athletic Shoe,8cb26a3e-7de4-4af3-ae40-272450fa9b4d,6.755055
8,Nike Men's 'Lunarglide 6' Synthetic Athletic Shoe,968a9319-fdd4-45ca-adc6-940cd83a204a,6.755055
9,Nike Ladies Pink Lunar Duet Sport Golf Shoes,c523a39a-82b1-4311-bf25-c572cb164a4b,6.402832


## Simple Vector Search

Let's prepare some functions to help us encode our data!


In [30]:
import torch
import clip
import requests
from PIL import Image

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

# First - let's encode the image based on CLIP
def encode_image(image):
    # Let us download the image and then preprocess it
    image = preprocess(Image.open(requests.get(image, stream=True).raw)).unsqueeze(0).to(device)
    # We then feed our processed image through the neural net to get a vector
    with torch.no_grad():
      image_features = model.encode_image(image)
    # Lastly we convert it to a list so that we can send it through the SDK
    return image_features.tolist()[0]

# Next - let's encode text based on CLIP
def encode_text(text):
    # let us get text and then tokenize it
    text = clip.tokenize([text]).to(device)
    # We then feed our processed text through the neural net to get a vector
    with torch.no_grad():
        text_features = model.encode_text(text)
    return text_features.tolist()[0]

100%|███████████████████████████████████████| 338M/338M [00:15<00:00, 22.7MiB/s]


In [34]:

# Encoding the query
query_vector = encode_text('nike')

results = ds.advanced_search(
    vector_search_query=[
    {
      "vector": query_vector,
      "field":'product_title_clip_vector_'
     }
], 
select_fields=['product_title'])

pd.DataFrame(results['results'])


,product_title,_id,_relevance
0,Nike Women's 'Son Of Force Low' Leather Athlet...,f0776d1d-58c2-40e1-a6a8-1389ab7c9097,0.693284
1,Classic Tote Bag,89f74212-e9fd-46da-90f0-157d54a93693,0.691830
2,Nike Men's 'Lunarglide 6' Synthetic Athletic Shoe,8cb26a3e-7de4-4af3-ae40-272450fa9b4d,0.690562
3,Nike Men's 'Air Max '93' Leather Athletic Shoe,d97d11df-0c37-4e33-8ac6-315e73884be0,0.690420
4,Nike Men's 'Lunarglide 6' Synthetic Athletic Shoe,968a9319-fdd4-45ca-adc6-940cd83a204a,0.685140
5,PS4 - Destiny,a5a6ee33-17da-4da8-b675-d18d4a43a6e4,0.683079
6,Panasonic Earbud Headphones,83d1f654-2a47-44e7-994d-dc1c48c9abc6,0.679970
7,Panasonic Earbud Headphones,ecd884ed-6acf-4bff-9dd4-d2ca1f82c4d6,0.679800
8,Panasonic Earbud Headphones,d51b8c05-b5b2-4667-b482-68f16a8fc7c6,0.679770
9,Panasonic Earbud Headphones,e694014a-f336-45d1-95a9-54ab55f676fc,0.679770


## Combining Text And Vector Search (Hybrid)

Combining text and vector search allows users get the best of both exact text search and contextual vector search. This can be done as shown below.

In [35]:

results = ds.advanced_search(
    query="nike",
    fields_to_search=["product_title"],
    vector_search_query=[
        {
        "vector": query_vector,
         "field":'product_title_clip_vector_'}
    ],
    select_fields = ["product_title"], # results to return
)

pd.DataFrame(results['results'])


,product_title,_id,_relevance
0,Nike Women's 'Lunaracer+ 3' Mesh Athletic Shoe,7baea34f-fb0a-47da-9edd-d920abddccf5,7.408601
1,Nike Air Men's Range WP Golf Shoes,e8d2552f-3ca5-4d15-9ca7-86855025b183,7.405841
2,Nike Ladies Lunar Duet Sport Golf Shoes,b655198b-4356-4ba9-b88e-1e1d6608f43e,7.358681
3,Nike Ladies Lunar Duet Sport Golf Shoes,80210247-6f40-45be-8279-8743b327f1dc,7.358681
4,Nike Mens Lunar Cypress Spikeless Golf Shoes,fb323476-a16d-439c-9380-0bac1e10a06d,7.329358
5,Nike Women's Lunar Duet Classic Golf Shoes,e1f3faf0-72fa-4559-9604-694699426cc2,7.314920
6,Nike Women's Lunar Duet Classic Golf Shoes,6f85d037-7621-45ee-b5dc-dd0e88c58d4a,7.314820
7,Nike SolarSoft Golf Grill Room Black Shoes,22871acd-fbc9-462e-8305-26df642c915c,7.280349
8,Nike Junior's Range Red/ White Golf Shoes,d27e70f3-2884-4490-9742-133166795d0f,7.264541
9,Nike Men's 'Air Max Pillar' Synthetic Athletic...,57ca8324-3e8a-4926-9333-b10599edb17b,7.136653


## Adjust the weighting of your vector search results

Adjust the weighting of your vector search results to make it easier for you!
Simply add a `weight` parameter your dictionary inside `vector_search_query`.

In [36]:

results = ds.advanced_search(
    query="nike",
    fields_to_search=["product_title"],
    vector_search_query=[
        {
          "vector": query_vector,
         "field":'product_title_clip_vector_', 
         "weight": 0.5
         }
    ],
    select_fields=["product_title"], # results to return
)

pd.DataFrame(results['results'])

,product_title,_id,_relevance
0,Nike Women's 'Lunaracer+ 3' Mesh Athletic Shoe,7baea34f-fb0a-47da-9edd-d920abddccf5,7.081828
1,Nike Air Men's Range WP Golf Shoes,e8d2552f-3ca5-4d15-9ca7-86855025b183,7.080448
2,Nike Ladies Lunar Duet Sport Golf Shoes,b655198b-4356-4ba9-b88e-1e1d6608f43e,7.056868
3,Nike Ladies Lunar Duet Sport Golf Shoes,80210247-6f40-45be-8279-8743b327f1dc,7.056868
4,Nike Mens Lunar Cypress Spikeless Golf Shoes,fb323476-a16d-439c-9380-0bac1e10a06d,7.042206
5,Nike Women's Lunar Duet Classic Golf Shoes,e1f3faf0-72fa-4559-9604-694699426cc2,7.034987
6,Nike Women's Lunar Duet Classic Golf Shoes,6f85d037-7621-45ee-b5dc-dd0e88c58d4a,7.034937
7,Nike SolarSoft Golf Grill Room Black Shoes,22871acd-fbc9-462e-8305-26df642c915c,7.017702
8,Nike Junior's Range Red/ White Golf Shoes,d27e70f3-2884-4490-9742-133166795d0f,7.009798
9,Nike Men's 'Air Max Pillar' Synthetic Athletic...,57ca8324-3e8a-4926-9333-b10599edb17b,6.769743


## Multi-Vector Search Across Multiple Fields

You can easily add more to your search by extending your vector search query as belows.

In [44]:

from PIL import Image
import requests
import numpy as np


image_url = 'https://static.nike.com/a/images/t_PDP_1280_v1/f_auto,q_auto:eco/e6ea66d1-fd36-4436-bcac-72ed14d8308d/wearallday-younger-shoes-5bnMmp.png'

image_vector = encode_image(image_url)

results = ds.advanced_search(
    query="nike",
    fields_to_search=["product_title"],
    vector_search_query=[
        {"vector": query_vector, "field": 'product_title_clip_vector_'},
        {"vector": image_vector, "field": 'product_image_clip_vector_'}
    ],
    select_fields=["product_title", "product_image", "product_price", "query"], # results to return
)

from relevanceai import show_json


display(
    show_json(
        results['results'],
        image_fields=['product_image'],
        text_fields=['query', 'product_title', 'product_price'],
    )
)

# pd.DataFrame(results['results'])


,product_image,query,product_title,product_price,_id
0,,nike shoes,Nike Air Men's Range WP Golf Shoes,$90.99 - $91.04,e8d2552f-3ca5-4d15-9ca7-86855025b183
1,,nike womens,Nike Ladies Lunar Duet Sport Golf Shoes,$81.99 - $88.07,80210247-6f40-45be-8279-8743b327f1dc
2,,nike shoes,Nike Ladies Lunar Duet Sport Golf Shoes,$81.99 - $88.07,b655198b-4356-4ba9-b88e-1e1d6608f43e
3,,nike shoes,Nike Mens Lunar Cypress Spikeless Golf Shoes,$100.99,fb323476-a16d-439c-9380-0bac1e10a06d
4,,nike shoes,Nike SolarSoft Golf Grill Room Black Shoes,$49.99,22871acd-fbc9-462e-8305-26df642c915c
5,,nike shoes,Nike Junior's Range Red/ White Golf Shoes,$49.99,d27e70f3-2884-4490-9742-133166795d0f
6,,nike womens,Nike Women's Lunar Duet Classic Golf Shoes,$97.99,6f85d037-7621-45ee-b5dc-dd0e88c58d4a
7,,nike shoes,Nike Women's Lunar Duet Classic Golf Shoes,$97.99,e1f3faf0-72fa-4559-9604-694699426cc2
8,,nike womens,Nike Women's SQ Dymo STR8-FIT Driver,$146.99,ff52b64a-0567-4181-8753-763da7044f2f
9,,nike shoes,Nike Mens Lunar Mont Royal Spikeless Golf Shoes,$100.99,e692a73b-a144-4e44-b4db-657be6db96e2
